Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [43]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from timeit import default_timer as timer
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'


def load_data(data):
    global test_dataset
    test_dataset = data[0][1]
    global test_labels
    test_labels = data[0][2]
    global valid_dataset
    valid_dataset = data[1][1]
    global valid_labels
    valid_labels = data[1][2]
    global train_dataset
    train_dataset = data[2][1]
    global train_labels
    train_labels = data[2][2]
    
def print_data_shape():
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

with open(pickle_file, 'rb') as f:
    sets = pickle.load(f)
    load_data(sets['unsanitized'])
    del sets  # hint to help gc free up memory
    print_data_shape()
    

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

def make_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def make_xavier_weights(shape, name):
    return tf.get_variable(name, shape, initializer=tf.contrib.layers.xavier_initializer())

def make_bias(shape, constant=0.1):
    return tf.Variable(tf.constant(constant, shape=shape))

iteration_log = []
def log_step(start_time, duration, progress, loss, batch_acc, valid_acc=False, test_acc=False):
    iteration_log.append([start_time, duration, progress, loss, batch_acc, valid_acc, test_acc])
    print('>>>>>', int(progress * 100), '% done')
    print("time to execute last %d minibatches: %f" % (int(num_steps / num_prints), duration))
    print()
    print("loss of last minibatch %f" % loss)
    print("accuracy on last minibatch: %.1f%%" % batch_acc)
    print()
    if (valid_acc):
        print("model's current accuracy against validation data : %.1f%%" % valid_acc)
    if (test_acc):
        print("model's current accuracy against test data       : %.1f%%" % test_acc)
    print()


Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder( tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal( [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal( [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal( [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal( [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy( valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.656298
Minibatch accuracy: 12.5%
Validation accuracy: 11.3%
Minibatch loss at step 50: 1.723300
Minibatch accuracy: 43.8%
Validation accuracy: 48.2%
Minibatch loss at step 100: 1.395713
Minibatch accuracy: 56.2%
Validation accuracy: 70.2%
Minibatch loss at step 150: 0.553474
Minibatch accuracy: 75.0%
Validation accuracy: 76.7%
Minibatch loss at step 200: 0.644388
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 250: 1.160439
Minibatch accuracy: 62.5%
Validation accuracy: 78.6%
Minibatch loss at step 300: 0.530756
Minibatch accuracy: 75.0%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.229902
Minibatch accuracy: 87.5%
Validation accuracy: 78.1%
Minibatch loss at step 400: 0.247715
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 450: 0.393775
Minibatch accuracy: 93.8%
Validation accuracy: 80.6%
Minibatch loss at step 500: 0.521885
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [11]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder( tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal( [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal( [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.max_pool(data, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [12]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy( valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.446263
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.372881
Minibatch accuracy: 6.2%
Validation accuracy: 16.5%
Minibatch loss at step 100: 2.163402
Minibatch accuracy: 12.5%
Validation accuracy: 11.5%
Minibatch loss at step 150: 1.955537
Minibatch accuracy: 31.2%
Validation accuracy: 18.0%
Minibatch loss at step 200: 2.464423
Minibatch accuracy: 18.8%
Validation accuracy: 19.3%
Minibatch loss at step 250: 2.593349
Minibatch accuracy: 12.5%
Validation accuracy: 15.8%
Minibatch loss at step 300: 1.684439
Minibatch accuracy: 37.5%
Validation accuracy: 40.5%
Minibatch loss at step 350: 1.745638
Minibatch accuracy: 56.2%
Validation accuracy: 35.8%
Minibatch loss at step 400: 1.178022
Minibatch accuracy: 56.2%
Validation accuracy: 48.7%
Minibatch loss at step 450: 1.287710
Minibatch accuracy: 62.5%
Validation accuracy: 45.2%
Minibatch loss at step 500: 1.490352
Minibatch accuracy: 50.0%
Validation accuracy: 56.8%
Min

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [57]:
batch_size = 128
num_relus = 1024
start_learning_rate = 0.1
beta_loss_coefficient = 0.001

batch_size = 64 
patch_size = 5
depth = [4, 6] 
num_hidden = 64

layer_shape = [image_size * image_size, 1024, 1024, 1024, num_labels]

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder( tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    
    weights = [
        make_xavier_weights([patch_size, patch_size, num_channels, depth[0]], '0_conv_layer'),
        make_xavier_weights([patch_size, patch_size, depth[0], depth[1]], '2_conv_layer'),
        make_xavier_weights([image_size // 4 * image_size // 4 * depth[1], num_hidden], '4_connected'),
        make_xavier_weights([num_hidden, num_labels], '5_connected'),
    ];
    biases = [
        make_bias([depth[0]]),
        make_bias([depth[1]]),
        make_bias([num_hidden]),
        make_bias([num_labels]),
    ];
    dropout = [
        .5,
#         .5,
        .5,
#         .5,
        .5
    ]
    
     
    def model(data, should_print=True):
        if (should_print): print('input: ', data.get_shape())
        conv = tf.nn.conv2d(data, weights[0], [1, 1, 1, 1], padding='SAME')
        if (should_print): print(conv.get_shape())
        subsample = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(subsample + biases[0])
        if (dropout[0] > 0 and dropout[0] < 1):
            hidden = tf.nn.dropout(hidden, dropout[0])
        if (should_print): print('layer 0')
        if (should_print): print(weights[0].get_shape())
        if (should_print): print(hidden.get_shape())
        
        conv = tf.nn.conv2d(hidden, weights[1], [1, 1, 1, 1], padding='SAME')
        subsample = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(subsample + biases[1])
        if (dropout[1] > 0 and dropout[1] < 1):
            hidden = tf.nn.dropout(hidden, dropout[1])
        if (should_print): print('layer 1')
        if (should_print): print(weights[1].get_shape())
        if (should_print): print(hidden.get_shape())
        
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        if (should_print): print(reshape.get_shape())
        
        
        connected = tf.matmul(reshape, weights[2])
        hidden = tf.nn.relu(connected + biases[2])
        if (dropout[2] > 0 and dropout[2] < 1):
            hidden = tf.nn.dropout(hidden, dropout[2])
        if (should_print): print('layer 2')
        if (should_print): print(weights[2].get_shape())
        if (should_print): print(hidden.get_shape())
        
        logits = tf.matmul(hidden, weights[3]) 
        print('logits: ', logits.get_shape())
        return logits + biases[3]
    
    
    logits = model(tf_train_dataset)
    
    unreduced_loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    for i in range(len(weights)):
        unreduced_loss = unreduced_loss + beta_loss_coefficient * tf.nn.l2_loss(weights[i])
        unreduced_loss = unreduced_loss + beta_loss_coefficient * tf.nn.l2_loss(biases[i])
    
    loss = tf.reduce_mean(unreduced_loss)
    
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(start_learning_rate, num_steps, 100000, 0.96, staircase=True)
    # Use simple momentum for the optimization.
#     optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=global_step)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))


input:  (64, 28, 28, 1)
(64, 28, 28, 4)
layer 0
(5, 5, 1, 4)
(64, 14, 14, 4)
layer 1
(5, 5, 4, 6)
(64, 7, 7, 6)
(64, 294)
layer 2
(294, 64)
(64, 64)
logits:  (64, 10)
input:  (10000, 28, 28, 1)
(10000, 28, 28, 4)
layer 0
(5, 5, 1, 4)
(10000, 14, 14, 4)
layer 1
(5, 5, 4, 6)
(10000, 7, 7, 6)
(10000, 294)
layer 2
(294, 64)
(10000, 64)
logits:  (10000, 10)
input:  (10000, 28, 28, 1)
(10000, 28, 28, 4)
layer 0
(5, 5, 1, 4)
(10000, 14, 14, 4)
layer 1
(5, 5, 4, 6)
(10000, 7, 7, 6)
(10000, 294)
layer 2
(294, 64)
(10000, 64)
logits:  (10000, 10)


In [59]:
num_steps = 5000
num_prints = 10
beta_loss_coefficient = .01

with tf.Session(graph=graph) as session:
    initial_time = timer()
    start = timer()
    tf.global_variables_initializer().run()
    print("Initialized")
    accuracies = [[],[],[]] # train, valid, test
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        accuracies[0].append(accuracy(predictions, batch_labels))
#         accuracies[1].append(accuracy(valid_prediction.eval(), valid_labels))
#         accuracies[2].append(accuracy(test_prediction.eval(), test_labels))
        
        if (step % int(num_steps / num_prints) == 0):
            end = timer()
            accuracies[1].append(accuracy(valid_prediction.eval(), valid_labels))
            accuracies[2].append(accuracy(test_prediction.eval(), test_labels))
            log_step(start, end - start, step / num_steps, l, accuracies[0][-1], accuracies[1][-1], accuracies[2][-1])
            start = timer()
    print()
    print("Total time for %d iterations: %d" % (num_steps, timer() - initial_time))


Initialized
>>>>> 0 % done
time to execute last 500 minibatches: 0.406662

loss of last minibatch 2.758705
accuracy on last minibatch: 14.1%

model's current accuracy against validation data : 11.4%
model's current accuracy against test data       : 11.3%

>>>>> 0 % done
time to execute last 500 minibatches: 10.292171

loss of last minibatch 1.051962
accuracy on last minibatch: 67.2%

model's current accuracy against validation data : 64.1%
model's current accuracy against test data       : 70.2%

>>>>> 0 % done
time to execute last 500 minibatches: 10.537870

loss of last minibatch 1.170667
accuracy on last minibatch: 59.4%

model's current accuracy against validation data : 69.1%
model's current accuracy against test data       : 77.0%

>>>>> 0 % done
time to execute last 500 minibatches: 10.518086

loss of last minibatch 0.862216
accuracy on last minibatch: 71.9%

model's current accuracy against validation data : 72.5%
model's current accuracy against test data       : 79.2%

>>>>>